In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import sktime
from sktime.utils.data_io import load_from_tsfile_to_dataframe
import data_loader as data

DATA_PATH = 'data/'
DATASET = 'RR'

In [41]:
X_train_orig, y_train_orig = data.load_from_tsfile_to_dataframe(
    os.path.join(f'{DATA_PATH}/BIDMC32{DATASET}_TRAIN.ts'),
    replace_missing_vals_with='NaN'
)

X_test_orig, y_test_orig = data.load_from_tsfile_to_dataframe(
    os.path.join(f'{DATA_PATH}/BIDMC32{DATASET}_TEST.ts'),
    replace_missing_vals_with='NaN'
)



5492it [00:46, 118.38it/s]
2420it [00:20, 120.81it/s]


In [42]:
print(X_train_orig.shape, type(X_train_orig))
print(y_train_orig.shape, type(y_train_orig))
print(X_test_orig.shape, type(X_test_orig))
print(y_test_orig.shape, type(y_test_orig))

(5471, 2) <class 'pandas.core.frame.DataFrame'>
(5471,) <class 'numpy.ndarray'>
(2399, 2) <class 'pandas.core.frame.DataFrame'>
(2399,) <class 'numpy.ndarray'>


In [43]:
y_test_orig.dtype

dtype('float64')

In [44]:
def split_data(X_train_orig, y_train_orig, X_test_orig, y_test_orig, shuffle=True, seed=42):
    if shuffle:
        X_all = pd.concat((X_train_orig, X_test_orig))
        y_all = np.concatenate((y_train_orig, y_test_orig))
        X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.15, random_state=seed)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=seed)
    else:
        X_train, y_train = X_train_orig, y_train_orig
        m = X_test_orig.shape[0] // 2
        X_val, X_test = X_test_orig.iloc[:m, :], X_test_orig.iloc[m:, :]
        y_val, y_test = y_test_orig.iloc[:m, :], y_test_orig.iloc[m:, :]
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = split_data(X_train_orig, y_train_orig, X_test_orig, y_test_orig, shuffle=True)



In [45]:
def _to_numpy(X):
    return np.stack([np.stack(x) for x in X.to_numpy()]).swapaxes(-1, -2)

np.save(os.path.join(DATA_PATH, DATASET, "trainx.npy"), _to_numpy(X_train))
np.save(os.path.join(DATA_PATH, DATASET, "trainy.npy"), y_train)
np.save(os.path.join(DATA_PATH, DATASET, "validx.npy"), _to_numpy(X_val)-1)
np.save(os.path.join(DATA_PATH, DATASET, "validy.npy"), y_val)
np.save(os.path.join(DATA_PATH, DATASET, "testx.npy"), _to_numpy(X_test))
np.save(os.path.join(DATA_PATH, DATASET, "testy.npy"), y_test)



In [46]:
for file in ['trainx', 'trainy', 'validx', 'validy', 'testx', 'testy']:
    a = np.load(f"{DATA_PATH}/{DATASET}/{file}.npy")
    print(a.shape)
    print(a.dtype)

(5685, 4000, 2)
float64
(5685,)
float64
(1004, 4000, 2)
float64
(1004,)
float64
(1181, 4000, 2)
float64
(1181,)
float64
